This tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2025.1_tutorials/wallaroo-run-anywhere/inference/arm/wallaroo-arm-byop-vgg16).

## Model Upload API Error Message Tests


## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.

In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import wallaroo

from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework
import base64
import requests
import time
import json

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more details on logging in through Wallaroo, see the [Wallaroo SDK Essentials Guide: Client Connection](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [10]:
wl = wallaroo.Client()



### Set Variables

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.



In [11]:
workspace_name = f'vgg16-clustering-workspace-error-api'


### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline that is used to deploy our arbitrary Python model.

In [12]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
wl.set_current_workspace(workspace)


{'name': 'vgg16-clustering-workspace-error-api', 'id': 1814, 'archived': False, 'created_by': '7d603858-88e0-472e-8f71-e41094afd7ec', 'created_at': '2025-12-18T21:54:59.44398+00:00', 'models': [], 'pipelines': [{'name': 'vgg16-clustering-error-tests', 'create_time': datetime.datetime(2025, 12, 18, 21, 54, 59, 717343, tzinfo=tzutc()), 'definition': '[]'}]}

### Test List

- API - Unmodified BYOP file
- API - requirements.txt that is empty
- API -  requirements.txt that is not at the root level
- API -  A .py file without an Inference class 
- API -  An Inference class without expected_model_types method
- API -  An Inference class without a predict method
- API -  A Predict method that does not accept a dictionary of numpy arrays
- API -  A Predict method that does not return a dictionary of numpy arrays
- API -  An Inference class that is missing a model method
- API -  A .py file without an InferenceBuilder class
- API -  An InferenceBuilder class without an inference method
- API -  An InferenceBuilder class without a create method
- API -  Package dependencies fail to install"

Define standard parameters

In [9]:
input_schema = pa.schema([
    pa.field('images', pa.list_(
        pa.list_(
            pa.list_(
                pa.int64(),
                list_size=3
            ),
            list_size=32
        ),
        list_size=32
    )),
])

output_schema = pa.schema([
    pa.field('predictions', pa.int64()),
])

encoded_input_schema = base64.b64encode(
                bytes(input_schema.serialize())
            ).decode("utf8")

encoded_output_schema = base64.b64encode(
                bytes(output_schema.serialize())
            ).decode("utf8")

framework = 'custom'

### Test 1:  Normal Model

Unmodified model to show it can upload successfully without issue.



In [10]:
workspace.id()

1814

In [17]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'api-byop-error-demo-unmodified'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_name, open('./models/byop-vgg-unmodified.zip','rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)

'Uploaded Model Name: api-byop-error-demo-unmodified.'

{'msg': 'Bad request: The provided file needs to be a .zip, .pt, .onnx, .h5, or .pkl file.',
 'code': 400}